# Requetes et visualisation de données à partir de l'API HubEAU
site : https://hubeau.eaufrance.fr/

In [1]:
import requests
import pandas as pd
import folium

fonction permettant de gérer les différentes requètes du notebook

In [2]:
def requete_api(url, params=None):
    try:
        if params is not None:
            reponse = requests.get(url, params=params)
        else:
            reponse = requests.get(url)
        # Lève une exception si la requête échoue
        reponse.raise_for_status()
        return reponse.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return None

## Test permettant d'interroger les analyse physico-chimique de l'API qualité des cours d'eau 
Lien : https://hubeau.eaufrance.fr/page/api-qualite-cours-deau

In [3]:
url = "https://hubeau.eaufrance.fr/api/v2/qualite_rivieres/analyse_pc"

params = {
    "libelle_commune": "Toulouse",
    "libelle_parametre": "Nitrates",
    "date_debut_prelevement": "2023-01-01",
    "date_fin_prelevement": "2024-01-01",
    "code_qualification": "1",
    "pretty": True,
    "sort": "desc",
    "fields": "longitude,latitude,libelle_station,libelle_parametre,date_prelevement,resultat,symbole_unite,mnemo_remarque,mnemo_statut,code_qualification"
}

data = requete_api(url, params=params)

if data is not None:
    df = pd.DataFrame(data)

### Visualisation des stations d'analyse

In [4]:
# Créer une carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=[43.602566, 1.397389], zoom_start=13).add_to(f)

# Ajouter SOLAGRO à la carte
folium.Marker([43.602566, 1.397389], popup='SOLAGRO', icon=folium.Icon(color='red')).add_to(m)

# dt.data is pandas.core.series.Series
lon = df.data.apply(lambda x: x['longitude']).values
lat = df.data.apply(lambda x: x['latitude']).values
libelle_station = df.data.apply(lambda x: x['libelle_station']).values

for i in range(len(lon)):
    folium.Marker(location=[lat[i], lon[i]], popup=libelle_station[i]).add_to(m)

display(m)

### Données brutes

In [5]:
from tabulate import tabulate
# if data is not None:
#     print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

print(df.data.values)

[{'libelle_station': 'PECH DAVID', 'date_prelevement': '2023-12-18', 'libelle_parametre': 'Nitrates', 'resultat': 5.6, 'symbole_unite': 'mg/L', 'mnemo_remarque': 'Résultat > seuil de quantification et < au seuil de saturation', 'mnemo_statut': 'Donnée contrôlée niveau 1 (données contrôlées)', 'code_qualification': '1', 'longitude': 1.419491118, 'latitude': 43.60934733}
 {'libelle_station': 'La Garonne à Blagnac', 'date_prelevement': '2023-12-13', 'libelle_parametre': 'Nitrates', 'resultat': 4.3, 'symbole_unite': 'mg(NO3)/L', 'mnemo_remarque': 'Résultat > seuil de quantification et < au seuil de saturation', 'mnemo_statut': 'Donnée contrôlée niveau 1 (données contrôlées)', 'code_qualification': '1', 'longitude': 1.400961721, 'latitude': 43.624924498}
 {'libelle_station': 'La Garonne au Syndicat Centre et Nord', 'date_prelevement': '2023-12-13', 'libelle_parametre': 'Nitrates', 'resultat': 5.2, 'symbole_unite': 'mg(NO3)/L', 'mnemo_remarque': 'Résultat > seuil de quantification et < au se

## Test permettant d'interroger les prélèvements en eau de l'API Prélèvements en eau

Lien : https://hubeau.eaufrance.fr/page/api-prelevements-eau

In [6]:
# url_prelev_eau = "https://hubeau.eaufrance.fr/api/v1/prelevements/referentiel/ouvrages?nom_commune=Toulouse&pretty"
url_prelev_eau = "https://hubeau.eaufrance.fr/api/v1/prelevements/chroniques"
# https://hubeau.eaufrance.fr/api/v1/prelevements/referentiel/ouvrages?nom_commune=Toulouse&pretty&fields=longitude,latitude,nom_ouvrage
params_prelev_eau = {
    "nom_commune": "Toulouse",
    "annee": 2022,
    "fields": "code_ouvrage,nom_ouvrage,longitude,latitude,annee,volume,code_mode_obtention_volume,code_usage"
}

data_prelev_eau = requete_api(url_prelev_eau, params=params_prelev_eau)

if data_prelev_eau is not None:
    df_prelev_eau = pd.DataFrame(data_prelev_eau)


### Visualisation des ouvrages de prélèvements

In [7]:
# Créer une carte
f = folium.Figure(width=800, height=800)
m2 = folium.Map(location=[43.602566, 1.397389], zoom_start=13).add_to(f)

# Ajouter SOLAGRO à la carte
folium.Marker([43.602566, 1.397389], popup='SOLAGRO', icon=folium.Icon(color='red')).add_to(m2)

lon = df_prelev_eau.data.apply(lambda x: x['longitude']).values
lat = df_prelev_eau.data.apply(lambda x: x['latitude']).values
libelle_station = df_prelev_eau.data.apply(lambda x: x['nom_ouvrage']).values

for i in range(len(lon)):
    folium.Marker(location=[lat[i], lon[i]], popup=libelle_station[i]).add_to(m2)

display(m2)

### Données brutes

In [8]:
print(df_prelev_eau.data.values)

[{'code_ouvrage': 'OPR0000013335', 'annee': 2022, 'volume': 252967.0, 'code_usage': 'IND', 'code_mode_obtention_volume': 'MED', 'longitude': 1.3986062175210023, 'latitude': 43.58194415603316, 'nom_ouvrage': "SOC. D'EXPLOITATION THERMIQUE DU MIRAIL"}
 {'code_ouvrage': 'OPR0000030021', 'annee': 2022, 'volume': 11253.0, 'code_usage': 'IND', 'code_mode_obtention_volume': 'MED', 'longitude': 1.3900005288508819, 'latitude': 43.611578232321754, 'nom_ouvrage': 'LA MERIDIONALE DES'}
 {'code_ouvrage': 'OPR0000030023', 'annee': 2022, 'volume': 9851.0, 'code_usage': 'IND', 'code_mode_obtention_volume': 'MED', 'longitude': 1.419486472864847, 'latitude': 43.60934960268563, 'nom_ouvrage': 'LA MERIDIONALE DES'}
 {'code_ouvrage': 'OPR0000030024', 'annee': 2022, 'volume': 9939.0, 'code_usage': 'IND', 'code_mode_obtention_volume': 'MED', 'longitude': 1.419486472864847, 'latitude': 43.60934960268563, 'nom_ouvrage': 'LA MERIDIONALE DES'}
 {'code_ouvrage': 'OPR0000030025', 'annee': 2022, 'volume': 2477.0, '

## Test permettant d'interroger les ventes/achats de pesticides de l'API ente et achat de produits phytopharmaceutiques

Lien : https://hubeau.eaufrance.fr/page/api-vente-et-achat-de-produits-phytopharmaceutiques#/

In [9]:
url_phyto = "https://hubeau.eaufrance.fr/api/v1/vente_achat_phyto/achats/produits"
params_phyto = {
    "type_territoire": "Zone postale",
    "code_territoire": "31000",
    "annee_min": 2020,
    "annee_max": 2024,
    "fields": "achat_etranger,annee,amm,code_territoire,eaj,quantite,unite"
}

data_phyto = requete_api(url_phyto, params=params_phyto)

if data_phyto is not None:
    df_phyto = pd.DataFrame(data_phyto)


In [10]:
df_phyto.data.values

array([{'annee': 2022, 'achat_etranger': 'Non', 'unite': 'l', 'eaj': 'Non', 'quantite': 10.0, 'code_territoire': '31000', 'amm': '2100041'},
       {'annee': 2022, 'achat_etranger': 'Non', 'unite': 'kg', 'eaj': 'Non', 'quantite': 0.2, 'code_territoire': '31000', 'amm': '2090057'},
       {'annee': 2022, 'achat_etranger': 'Non', 'unite': 'kg', 'eaj': 'Non', 'quantite': 0.6, 'code_territoire': '31000', 'amm': '2110063'},
       {'annee': 2022, 'achat_etranger': 'Non', 'unite': 'l', 'eaj': 'Non', 'quantite': 31.5, 'code_territoire': '31000', 'amm': '2190509'},
       {'annee': 2022, 'achat_etranger': 'Non', 'unite': 'l', 'eaj': 'Non', 'quantite': 80.0, 'code_territoire': '31000', 'amm': '2090064'},
       {'annee': 2022, 'achat_etranger': 'Non', 'unite': 'l', 'eaj': 'Non', 'quantite': 12.2, 'code_territoire': '31000', 'amm': '2130152'},
       {'annee': 2022, 'achat_etranger': 'Non', 'unite': 'l', 'eaj': 'Non', 'quantite': 5.0, 'code_territoire': '31000', 'amm': '2110101'},
       {'annee